<a href="https://colab.research.google.com/github/jingche120/0126/blob/main/%E3%80%90Demo07c%E3%80%91AI%E4%BB%A3%E7%90%86%E8%A8%AD%E8%A8%88%E6%A8%A1%E5%BC%8F_%E5%93%A1%E7%91%9B%E5%BC%8F%E6%80%9D%E8%80%83%E7%94%9F%E6%88%90%E5%99%A8Two_Stage_CoT%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 🎯 任務說明：將「多慧式思考產生器」改寫為 Chain-of-Thought（CoT）推理過程

**原始任務：** 輸入一件小事或倒楣事，生成一段超級正向的社群貼文。

#### ✅ CoT 改寫版本流程：
1. **第一階段（思考階段）**：請 LLM 思考五種「為什麼這是一件超級幸運的事」的原因
2. **第二階段（產文階段）**：從這五個原因中挑出最有趣的一個，寫成社群發文（員瑛式風格 + 第一人稱 + emoji + "完全是 Lucky Vicky 呀!" 結尾）

這就是典型的 Planning 模式應用：先拆解、後執行。

#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [ ]:
import os
from google.colab import userdata

In [ ]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "openai/gpt-oss-120b"

In [ ]:
!pip install aisuite[all]

### 2. 使用 AISuite 的準備

In [ ]:
import aisuite as ai

In [ ]:
provider_planner = "groq"
model_planner="openai/gpt-oss-120b"

provider_writer = "groq"
model_writer = "openai/gpt-oss-120b"

# provider_planner = "groq"
# model_planner = "llama-3.3-70b-versatile"

# provider_writer = "groq"
# model_writer = "llama-3.3-70b-versatile"

#provider_reviewer = "openai"
#model_reviewer = "gpt-4o"

In [ ]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="openai/gpt-oss-120b"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 打造二階段

In [ ]:
system_planner = "請用台灣習慣的中文回應: 你是一位正向思考的大師, 擅長將預言未來。請針對使用者的事件，思考出五種『未來的產品』的原因, 列出條列式清單。用第一人稱思考也可以。"
system_writer = "請用台灣習慣的中文回應: 你是一位超級樂觀的社群貼文小天使，風格活潑好笑，善於用正向觀點撰寫貼文。請根據一個特別有趣的理由，寫出一段 Instagram 發文口吻的第一人稱貼文。結尾一定要說：『未來是充滿希望的呀！』而且中間要加入 emoji。"

In [ ]:
def lucky_post(prompt):
    # Step 1: CoT - 思考五個未來的產品
    planning_prompt = f"使用者說：{prompt}。請幫我想五個未來的產品。"
    lucky_reasons = reply(system_planner, planning_prompt,
                          provider = provider_planner,
                          model = model_planner
                          )

    # Step 2: 選出最有趣一項，寫成貼文
    generation_prompt = f"這是我想到的五個產品：\n{lucky_reasons}\n\n請從中選出一個最有趣的，然後根據它寫一段多慧式發文。"
    final_post = reply(system_writer, generation_prompt,
                       provider = provider_writer,
                       model = model_writer
                       )

    return lucky_reasons, final_post

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("### 🍀 多慧式思考產生器 Lucky Vicky 🌈")
    gr.Markdown("請輸入一件今天發生的事請，讓 AI 幫你想出點子！")
    user_input = gr.Textbox(label="今天發生的事情是…")
    btn = gr.Button("生成 Lucky Vicky 的神級貼文 ✨")

    with gr.Row():
        out1 = gr.Textbox(label="🧠 五個超幸運的理由（Planning CoT）")
        out2 = gr.Textbox(label="📣 最終多慧式貼文（Po 文）")

    btn.click(lucky_post, inputs=[user_input], outputs=[out1, out2])

In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6aeda3e77cf88e9d75.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^